In [ ]:
# ==================== 导入必要的库 ====================
# torch: PyTorch深度学习框架
import torch

# nn: PyTorch的神经网络模块，包含各种层和损失函数
from torch import nn

# d2l: Dive into Deep Learning工具库
from d2l import torch as d2l

In [ ]:
# ==================== 转置卷积的基础实现 ====================

def trans_conv(X, K):
    """
    转置卷积（反卷积）的基础实现
    
    什么是转置卷积？
    - 转置卷积是卷积的逆运算，用于上采样（增大特征图尺寸）
    - 在语义分割中，需要将小的特征图放大到原图大小
    - 转置卷积通过在输入间插入零值，然后进行卷积来实现放大
    
    工作原理：
    - 对输入X的每个元素，与卷积核K相乘
    - 将结果放置在输出的对应位置
    - 有重叠的地方进行累加
    
    参数:
        X: 输入特征图，形状为 (height, width)
        K: 卷积核，形状为 (kernel_h, kernel_w)
    
    返回:
        Y: 输出特征图，形状为 (height+kernel_h-1, width+kernel_w-1)
    
    举例：如果输入是2x2，卷积核是2x2，输出将是3x3
    """
    h, w = K.shape  # 获取卷积核的高度和宽度
    
    # 创建输出张量，尺寸比输入大
    # 输出的高度 = 输入高度 + 卷积核高度 - 1
    # 输出的宽度 = 输入宽度 + 卷积核宽度 - 1
    Y = torch.zeros((X.shape[0] + h - 1, X.shape[1] + w - 1))
    
    # 遍历输入的每个元素
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            # 将输入元素X[i,j]与整个卷积核相乘
            # 然后将结果加到输出Y的对应区域
            # 这就是为什么叫"转置"卷积：输入的每个值影响输出的一个区域
            Y[i: i + h, j: j + w] += X[i, j] * K
    
    return Y

In [ ]:
# ==================== 测试转置卷积 ====================

# 创建一个2x2的输入矩阵
X = torch.tensor([[0.0, 1.0], [2.0, 3.0]])

# 创建一个2x2的卷积核
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])

# 执行转置卷积
# 输入是2x2，卷积核是2x2，输出将是3x3
# 观察输出如何从小的特征图生成大的特征图
trans_conv(X, K)

In [ ]:
# ==================== 使用PyTorch内置的转置卷积层 ====================

# 将输入和卷积核reshape为4D张量
# PyTorch的卷积层需要4D输入：(batch_size, channels, height, width)
# reshape(1, 1, 2, 2) 表示：1个样本，1个通道，2x2的特征图
X, K = X.reshape(1, 1, 2, 2), K.reshape(1, 1, 2, 2)

# 创建转置卷积层
# 参数说明：
#   1: 输入通道数
#   1: 输出通道数
#   kernel_size=2: 卷积核大小为2x2
#   bias=False: 不使用偏置项
tconv = nn.ConvTranspose2d(1, 1, kernel_size=2, bias=False)

# 将我们自定义的卷积核赋值给层的权重
# 这样可以验证PyTorch的实现与我们的手动实现是否一致
tconv.weight.data = K

# 执行转置卷积
# 输出应该与我们手动实现的trans_conv(X, K)结果相同
tconv(X)

In [ ]:
# ==================== 转置卷积中的填充 ====================

# 创建带填充的转置卷积层
# padding=1: 在输出的四周各去掉1个像素
# 
# 填充的作用：
# - 在普通卷积中，padding增加输入边界
# - 在转置卷积中，padding减少输出尺寸
# - padding=1会从输出的每边去掉1行/列
tconv = nn.ConvTranspose2d(1, 1, kernel_size=2, padding=1, bias=False)
tconv.weight.data = K

# 执行带填充的转置卷积
# 对比：不带padding时输出是3x3
#       带padding=1时，输出变为1x1（从3x3的每边去掉1）
tconv(X)

In [ ]:
# ==================== 卷积和转置卷积的可逆性验证 ====================

# 创建一个随机输入：1个样本，10个通道，16x16的特征图
X = torch.rand(size=(1, 10, 16, 16))

# 创建普通卷积层
# 10个输入通道 -> 20个输出通道
# kernel_size=5: 5x5的卷积核
# padding=2: 填充2个像素（保持空间尺寸）
# stride=3: 步幅为3（每3个像素移动一次，缩小特征图）
conv = nn.Conv2d(10, 20, kernel_size=5, padding=2, stride=3)

# 创建对应的转置卷积层
# 20个输入通道 -> 10个输出通道（与conv相反）
# 使用相同的kernel_size、padding、stride
# 目的：尝试恢复原始尺寸
tconv = nn.ConvTranspose2d(20, 10, kernel_size=5, padding=2, stride=3)

# 验证：先卷积再转置卷积，输出形状是否与原始输入相同
# conv(X): 将X下采样
# tconv(conv(X)): 再上采样回去
# 如果设置得当，形状应该恢复到原始大小
# 注意：形状相同不代表值相同，只是尺寸恢复了
tconv(conv(X)).shape == X.shape

In [ ]:
# ==================== 卷积与矩阵变换的关系 ====================

# 创建一个3x3的输入矩阵
X = torch.arange(9.0).reshape(3, 3)

# 创建一个2x2的卷积核
K = torch.tensor([[1.0, 2.0], [3.0, 4.0]])

# 使用d2l的corr2d函数执行2D互相关运算（卷积）
# 3x3的输入，2x2的卷积核 -> 2x2的输出
Y = d2l.corr2d(X, K)
print(Y)

# 这部分展示了卷积操作实际上可以表示为矩阵乘法
# 这对理解转置卷积很重要

In [ ]:
# ==================== 将卷积核转换为矩阵形式 ====================

def kernel2matrix(K):
    """
    将2D卷积核转换为矩阵形式
    
    为什么要这样做？
    - 卷积操作可以表示为矩阵乘法
    - 这有助于理解转置卷积的数学原理
    - 如果卷积是 Y = W*X，那么转置卷积就是 Z = W^T*Y
    
    参数:
        K: 2x2的卷积核
    
    返回:
        W: 4x9的权重矩阵，可以通过矩阵乘法实现卷积
    """
    # k: 临时向量，存储展开的卷积核
    # W: 权重矩阵，每一行对应输出的一个元素
    k, W = torch.zeros(5), torch.zeros((4, 9))
    
    # 将2x2卷积核的值填充到长度为5的向量中（中间留一个0）
    # K[0, :] 是卷积核第一行 [1.0, 2.0]
    # K[1, :] 是卷积核第二行 [3.0, 4.0]
    k[:2], k[3:5] = K[0, :], K[1, :]
    
    # 构造4x9的权重矩阵
    # 每一行代表输出的一个位置
    # 每一行包含了该位置对应的卷积核在输入中的布局
    # 这4行对应2x2输出的4个位置
    W[0, :5], W[1, 1:6], W[2, 3:8], W[3, 4:] = k, k, k, k
    
    return W

# 将卷积核转换为矩阵
W = kernel2matrix(K)
print(W)
# W的每一行代表了卷积核在输入上滑动时的一个位置

In [ ]:
# ==================== 验证卷积等价于矩阵乘法 ====================

# 验证：卷积操作 Y = corr2d(X, K) 等价于矩阵乘法 Y = W * X
# 
# 步骤：
# 1. X.reshape(-1): 将3x3的X展平为9x1的向量
# 2. torch.matmul(W, X.reshape(-1)): 4x9矩阵乘以9x1向量 = 4x1向量
# 3. .reshape(2, 2): 将4x1向量重塑为2x2矩阵
# 4. 比较结果是否与Y相同
print(Y == torch.matmul(W, X.reshape(-1)).reshape(2, 2))

# 如果输出全为True，说明卷积确实可以用矩阵乘法表示
# 这是理解转置卷积的关键

In [ ]:
# ==================== 验证转置卷积等价于转置矩阵乘法 ====================

# 使用我们实现的trans_conv函数对Y进行转置卷积
# 输入Y是2x2，卷积核K是2x2，输出Z是3x3
Z = trans_conv(Y, K)

# 验证：转置卷积 Z = trans_conv(Y, K) 等价于 Z = W^T * Y
# 
# 关键理解：
# - 如果卷积是 Y = W * X（从大到小）
# - 那么转置卷积就是 Z = W^T * Y（从小到大）
# - W.T 是W的转置矩阵
# 
# 步骤：
# 1. Y.reshape(-1): 将2x2的Y展平为4x1向量
# 2. torch.matmul(W.T, Y.reshape(-1)): 9x4矩阵乘以4x1向量 = 9x1向量
# 3. .reshape(3, 3): 将9x1向量重塑为3x3矩阵
# 4. 比较结果是否与Z相同
print(Z == torch.matmul(W.T, Y.reshape(-1)).reshape(3, 3))

# 如果输出全为True，说明：
# 1. 转置卷积确实是矩阵转置乘法
# 2. 转置卷积可以看作是卷积的"逆操作"（在形状上）
# 3. 这就是为什么它能用于上采样和语义分割